In [ ]:
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Lambda
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import metrics
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import Counter
import tensorflow as tf
import scipy.stats
import pandas as pd
from tensorflow.contrib.metrics import streaming_pearson_correlation
from keras.models import load_model
from functools import partial
from sklearn.preprocessing import maxabs_scale, minmax_scale, normalize, scale, robust_scale

In [ ]:
import cobra
%load_ext autoreload
import sys
if not '/home/nlarusstone/cf_fba' in sys.path:
    sys.path.append('/home/nlarusstone/cf_fba')
import src.utils as utils
import src.flux_utils as futils
import src.create_dataset as dataset
import src.flux_sample as fs
%autoreload 2

In [ ]:
df = fs.get_exp_data('../data/{0}_data.CSV')
n_experiments = df.shape[0]

In [ ]:
y_vals = futils.scale_data(data=df['AVG.1'].values, scale_type='flux_zero', in_place=False)

In [ ]:
X_train, y_train, X_test, y_test, obj_col, cols = futils.read_data('../data/f2000', 'karim_karim_ecoli_cf_base.sbml_fluxes',
                                                                    n_experiments, 'DM_btol_c', n_rows=50000, scale='flux_zero')

In [ ]:
froot = 'hand'
txtl = False
resamp = True
fname = '../data/{0}{1}_{2}_fluxes'.format(froot, '_txtl' if txtl else '', 'stacked' if resamp else 'flat')
X_train, y_train, X_test, y_test, obj_col, cols, y_vals_d = dataset.get_dataset(fname)
y_vals = np.array(y_vals_d)

In [ ]:
#np.savez_compressed('../data/fluxes_resampled', train=X_train, test=X_test)
#dat = np.load('../data/fluxes_resampled.npz')

In [ ]:
latent_dim = 2
epsilon_std = 1.0

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], n_experiments, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

def corr_loss_np(y_true, y_pred):
    cent_pred = y_pred - np.mean(y_pred)
    cent_tr = y_true - np.mean(y_true)

    std_pred = np.std(y_pred)
    std_tr = np.std(y_true)

    return np.mean(cent_pred*cent_tr)/(std_pred*std_tr)

def corr_loss(y_true, y_pred):
    cent_pred = y_pred - K.mean(y_pred)
    cent_tr = y_true - K.mean(y_true)

    std_pred = K.std(y_pred)
    std_tr = K.std(y_true)
    return K.mean(cent_pred*cent_tr)/(std_pred*std_tr)

# y_true, y_pred
def gen_vae_loss(y_true, x_decoded_mean, z_log_var, z_mean):
    output_flux = x_decoded_mean[:, :, output_ind]
    #experiment_loss = scipy.stats.spearmanr(targets, output_flux)
    #experiment_loss = pearson_corr(output_flux, targets)#streaming_pearson_correlation(output_flux, targets)
    xent_loss = X_shape * metrics.mean_squared_error(y_true, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(xent_loss + kl_loss)# + experiment_loss
    return vae_loss

def kl_loss(z_log_var, z_mean):
    return - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)

def build_vae(X_shape, n_experiments, targets, output_ind, corr_impt, batch_size=100):
    encoded_dim1 = 1024
    encoded_sz = 256
    # Encoder network
    x = Input(shape=(n_experiments, X_shape,))
    #h = Dense(encoded_dim1, activation='relu')(x)
    h = Dense(encoded_sz, activation='relu')(x)#h)
    z_mean = Dense(latent_dim, name='z_mean')(h)
    z_log_var = Dense(latent_dim, name='z_log_var')(h)
    
    # Sample points from latent space
    z = Lambda(sampling, output_shape=(n_experiments,latent_dim,))([z_mean, z_log_var])
    
    # Decoder network
    decoder_h = Dense(encoded_sz, activation='relu')
    #decoder_h2 = Dense(encoded_dim1, activation='relu')
    decoder_mean = Dense(X_shape, activation='tanh')
    h_decoded = decoder_h(z)
    #h_decoded2 = decoder_h2(h_decoded)
    x_decoded_mean = decoder_mean(h_decoded)#2)

    # end-to-end autoencoder
    vae = Model(x, x_decoded_mean)
    #vae = Model(x, [x_decoded_mean, x_decoded_mean])
    output_flux = x_decoded_mean[:, :, output_ind]
    #experiment_loss = scipy.stats.spearmanr(targets, output_flux)
    experiment_loss_val = -1 * corr_loss(targets, output_flux)#streaming_pearson_correlation(output_flux, targets)
    xent_loss = x.shape[-1].value * metrics.mean_squared_error(x, x_decoded_mean)
    kl_loss_val = kl_loss(z_log_var, z_mean)
    vae_loss = K.mean(xent_loss + kl_loss_val) + corr_impt * experiment_loss_val
    #print x.shape, x_decoded_mean.shape, z_mean.shape, z_log_var.shape
    #print xent_loss.shape, kl_loss_val.shape, experiment_loss_val.shape, vae_loss.shape
    #vae_loss = K.sum(vae_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='rmsprop')
    #vae.compile(optimizer='rmsprop', loss=[lambda x, x_pred: gen_vae_loss(x, x_pred, z_log_var, z_mean),
    #                                       lambda x, x_pred: corr_loss(targets, x_pred[:, :, output_ind])])
    vae.summary()

    # encoder, from inputs to latent space
    encoder = Model(x, z_mean)

    # generator, from latent space to reconstructed inputs
    decoder_input = Input(shape=(n_experiments, latent_dim,))
    _h_decoded = decoder_h(decoder_input)
    #_h_decoded2 = decoder_h2(_h_decoded)
    _x_decoded_mean = decoder_mean(_h_decoded)#2)
    generator = Model(decoder_input, _x_decoded_mean)
    return vae, encoder, generator

In [ ]:
from keras.callbacks import Callback
from sklearn.metrics import mean_squared_error as mse
class LossHistory(Callback):
    def __init__(self):
        self.recon_losses = []
        self.kl_losses = []
        self.corr_losses = []
        
    def on_epoch_end(self, epoch, logs={}):
        y_true = self.validation_data[0]
        y_pred = self.model.predict(self.validation_data[0])
        corr_loss = -1 * corr_loss_np(y_vals, y_pred[:, :, obj_col.value])
        xent_loss = y_true.shape[-1] * np.mean(np.square(y_true - y_pred), axis=-1)
        inputs = [K.learning_phase()] + self.model.inputs
        zvar = K.function(inputs=inputs, outputs=[self.model.get_layer('z_log_var').output])
        zmn = K.function(inputs=inputs, outputs=[self.model.get_layer('z_mean').output])
        z_log_var = zvar([0, self.validation_data[0]])[0]
        z_mean = zmn([0, self.validation_data[0]])[0]
        kl_loss = - 0.5 * np.sum(1 + z_log_var - np.square(z_mean) - np.exp(z_log_var), axis=-1)
        print "Reconstruction loss: {0}".format(np.mean(xent_loss))
        print "KL loss: {0}".format(np.mean(kl_loss))
        print "Corr loss: {0}".format(corr_loss)
        self.recon_losses.append(np.mean(xent_loss))
        self.kl_losses.append(np.mean(kl_loss))
        self.corr_losses.append(corr_loss)

In [ ]:
X_train.shape

In [ ]:
#%%debug
batch_size = 256
X_shape, n_experiments = X_train.shape[2], len(y_vals)
targets = tf.convert_to_tensor(y_vals, dtype=tf.float32)
corr_impt = 2
vae, encoder, generator = build_vae(X_shape, n_experiments, targets, obj_col.value, corr_impt, batch_size)
es = EarlyStopping(patience=5)
lh = LossHistory()
#with tf.Session(config=tf.ConfigProto(
#                    intra_op_parallelism_threads=32)) as sess:
#    K.set_session(sess)
hist = vae.fit(X_train,
        shuffle='batch',
        epochs=20,
        batch_size=batch_size,
        validation_data=(X_test, None),
        callbacks=[es, lh])
#encoder.save('encoder_{0}.h5'.format(scale))
#generator.save('generator{0}.h5'.format(scale))

In [ ]:
%run ../src/vae.py -d 2 -b 256 -n 100 --layers 1024 1024 1024 --resamp --no-txtl -f karim

In [ ]:
lh.corr_losses

In [ ]:
with open('hi2', 'r') as f:
    b = pickle.load(f)

In [ ]:
X_train, y_train, X_test, y_test, btol_col, cols = futils.read_data('../data/flux_samps_2k', scale='flux_max_abs')

In [ ]:
np.min(X_train)

In [ ]:
with open('hi2', 'w') as f:
    pickle.dump(file=f, obj={'recon_losses': lh.recon_losses, 'kl_losses': lh.kl_losses, 'corr_losses': lh.corr_losses})

In [ ]:
x_test_encoded = encoder.predict(X_test, batch_size=batch_size)
x_test_gen = generator.predict(x_test_encoded, batch_size=batch_size)
#x_test_encoded

In [ ]:
def check_corr(gen_fluxes, df, btol_col):
    corrs = []
    for i in range(gen_fluxes.shape[0]):
        corr = scipy.stats.pearsonr(gen_fluxes[i, :, btol_col], df['AVG.1'])
        corrs.append(corr)
    mn = np.mean(corrs, axis=0)
    print mn
    print corrs[:5]
    return mn[0]
check_corr(x_test_gen, df, btol_col)

In [ ]:
def get_rct(df, rct, y_test):
    y_new = []
    for ind in y_test:
        y_new.append(df[rct][ind])
    return y_new
#get_rct(df, 'Glucose', y_test)

In [ ]:
#cm1 = cm.get_cmap('tab20b', 20)
#cm2 = cm.get_cmap('tab20c', 20)
cmap = cm.get_cmap('plasma', 41)
#cmap = lambda x: cm1(x) if x < 21 else cm2(x)
#for j in range(41):
j = 0
xmin, xmax = np.amin(x_test_encoded[:, j, 0]), np.amax(x_test_encoded[:, j, 0])
ymin, ymax = np.amin(x_test_encoded[:, j, 1]), np.amax(x_test_encoded[:, j, 1])
x_diff = (xmax - xmin) / 10.0
y_diff = (ymax - ymin) / 10.0
for col in df.columns[4:]:
    plt.figure(figsize=(10, 10))
    plt.scatter(x_test_encoded[:, j, 0], x_test_encoded[:, j, 1], c=get_rct(df, col, y_test), cmap=cmap)
    plt.xlim((xmin - x_diff, xmax + x_diff))
    plt.ylim((ymin - y_diff, ymax + y_diff))
    plt.title(col)
    plt.colorbar()
    plt.show()

plt.figure(figsize=(10, 10))
plt.scatter(x_test_encoded[:, j, 0], x_test_encoded[:, j, 1], c=y_test, cmap=cmap)
plt.xlim((xmin - x_diff, xmax + x_diff))
plt.ylim((ymin - y_diff, ymax + y_diff))
plt.title('Variant')
plt.colorbar()
plt.show()

In [ ]:
x_test_encoded.shape

In [ ]:
x_test_dim_1, x_test_dim_2 = np.mean(x_test_encoded, axis=1)[:, :]

In [ ]:
#cm1 = cm.get_cmap('tab20b', 20)
#cm2 = cm.get_cmap('tab20c', 20)
cmap = cm.get_cmap('plasma', 41)
#cmap = lambda x: cm1(x) if x < 21 else cm2(x)
#for j in range(41):
x_test_encoded_agg = np.mean(x_test_encoded, axis=1)
x_test_dim_1, x_test_dim_2 = x_test_encoded_agg[:, 0], x_test_encoded_agg[:, 1]
xmin, xmax = np.amin(x_test_dim_1), np.amax(x_test_dim_2)
ymin, ymax = np.amin(x_test_dim_1), np.amax(x_test_dim_2)
x_diff = (xmax - xmin) / 10.0
y_diff = (ymax - ymin) / 10.0
for col in df.columns[4:]:
    plt.figure(figsize=(10, 10))
    plt.scatter(x_test_dim_1, x_test_dim_2, c=get_rct(df, col, y_test), cmap=cmap)
    plt.xlim((xmin - x_diff, xmax + x_diff))
    plt.ylim((ymin - y_diff, ymax + y_diff))
    plt.title(col)
    plt.colorbar()
    plt.show()

In [ ]:
std_trials = np.std(flat_data, axis=1)
std_fluxes = np.std(flat_data, axis=0)
plt.hist(std_trials, label='Experiments')
plt.title('STD Devs across experiments')
plt.hist(std_fluxes, label='Fluxes')
plt.title('STD Devs across fluxes')
plt.xlabel('Std dev')
plt.legend()
plt.show()

In [ ]:
sor_flux = np.argsort(std_fluxes)

In [ ]:
sor_flux

In [ ]:
check_corr(biased_resamp_data, df, btol_col)

In [ ]:
enc = load_model('../models/encoder_epochs=300_batch=256_dimension=2_corr=True.h5')
gen = load_model('../models/generator_epochs=300_batch=256_dimension=2_corr=True.h5')

In [ ]:
enc_bad = load_model('../models/encoder_epochs=100_batch=256_dimension=2_corr=False.h5')
gen_bad = load_model('../models/generator_epochs=100_batch=256_dimension=2_corr=False.h5')

In [ ]:
enc_good = load_model('../models/encoder_epochs=100_batch=256_dimension=10_corr=True.h5')
gen_good = load_model('../models/generator_epochs=100_batch=256_dimension=10_corr=True.h5')

In [ ]:
def pred(biased_resamp_data, enc, gen):
    encoded_biased = enc.predict(biased_resamp_data)
    decoded_biased = gen.predict(encoded_biased)
    return check_corr(decoded_biased, df, btol_col)
pred(biased_resamp_data, enc, gen)
pred(biased_resamp_data, enc2, gen2)

In [ ]:
def add_noise(biased_resamp_data, enc, gen):
    noise_arr = np.logspace(start=-5, stop=1, num=10)
    corrs = []
    for noise in noise_arr:
        noisy_data = biased_resamp_data.copy()
        for i in range(n_experiments):
            s = noisy_data[:, i, :].shape
            noisy_data[:, i, :] += np.random.normal(scale=noise, size=s)
            #noisy_data[:, i, :] = minmax_scale(noisy_data[:, i, :])
        #scaled_noisy_data = scale_by_flux(noisy_data)
        corr = pred(noisy_data, enc, gen)
        corrs.append(corr)
    return zip(noise_arr, corrs) + [(0, pred(biased_resamp_data, enc, gen))]
#noise_res = add_noise(biased_resamp_data, enc, gen)
#noise_res2 = add_noise(biased_resamp_data, enc_good, gen_good)
noise_res3 = add_noise(biased_resamp_data, enc_bad, gen_bad)

In [ ]:
orig_corr = check_corr(biased_resamp_data, df, btol_col)
def plt_noise_corr(noise_data, orig_corr, ndim=2):
    orig_enc, noise_data = noise_data[-1], noise_data[:-1]
    plt.figure(figsize=(10, 8))
    plt.title('Latent dimension = {0}'.format(ndim))
    plt.axhline(y=-1 * orig_enc[1] if orig_enc[1] < 0 else orig_enc[1], label='Original data encoded')
    plt.axhline(y=orig_corr, label='Original data correlation', c='g')
    for noise, corr in noise_data:
        plt.scatter(x=noise, y=-1 * corr if corr < 0 else corr)
        plt.xlabel('Noise amount')
        plt.ylabel('Correlation')
    plt.xscale('log')
    plt.legend()
    plt.show()
plt_noise_corr(noise_res, orig_corr, ndim=2)
plt_noise_corr(noise_res2, orig_corr, ndim=10)
plt_noise_corr(noise_res3, orig_corr, ndim='2, no correlation loss')

In [ ]:
test_enc = enc.predict(X_test)
test_dec = gen.predict(test_enc)
dec_df = pd.DataFrame(data=test_dec[:, 0, :], columns=cols)

In [ ]:
dec_df.std()

In [ ]:
bad_cols = cols[dec_df.mean() < 0.001]
bad_cols

In [ ]:
import cobra
%load_ext autoreload
import sys
if not '/home/nlarusstone/cf_fba' in sys.path:
    sys.path.append('/home/nlarusstone/cf_fba')
import src.utils as utils
%autoreload 2

In [ ]:
model = cobra.io.read_sbml_model('../models/ecoli_cf_base.sbml')

In [ ]:
objs = []
for i, row in df.iterrows():
    print i
    model_i = utils.add_reagents_to_model(model, row)
    sol = model_i.optimize()
    objs.append(sol.objective_value)
scipy.stats.pearsonr(objs, df['AVG.1'])

In [ ]:
thresh = np.logspace(start=-4, stop=0, num=20)
for t in thresh:
    objs = []
    for i, row in df.iterrows():
        model_i = utils.add_reagents_to_model(model, row)
        dec_df = pd.DataFrame(data=test_dec[:, i, :], columns=cols)
        bad_cols = cols[dec_df.mean() < t]
        model_i.remove_reactions(bad_cols)
        sol = model_i.optimize()
        objs.append(sol.objective_value)
    print t, scipy.stats.pearsonr(objs, df['AVG.1'])

In [ ]:
scipy.stats.pearsonr(objs, df['AVG.1'])

In [ ]:
objs = []
for i, row in df.iterrows():
    print i
    model_i = utils.add_reagents_to_model(model, row)
    dec_df_g = pd.DataFrame(data=test_dec_good[:, i, :], columns=cols)
    bad_cols = cols[dec_df_g.mean() < 0.01]
    print len(bad_cols)
    model_i.remove_reactions(bad_cols)
    sol = model_i.optimize()
    objs.append(sol.objective_value)
scipy.stats.pearsonr(objs, df['AVG.1'])

In [ ]:
test_enc_good = enc_good.predict(X_test)
test_dec_good = gen_good.predict(test_enc)
dec_df_good = pd.DataFrame(data=test_dec[:, 0, :], columns=cols)

In [ ]:
bad_cols_2 = cols[dec_df_good.mean() < 0.001]
bad_cols_2

In [ ]:
bad_cols_2.isin(bad_cols)